
# GWR in BDT Using Uber H3

Presented By: Alex Almond, Software Developer for Big Data Toolkit (BDT)

In [1]:
import bdt
bdt.auth("../bdt.lic")
from bdt.functions import *
from bdt.processors import *

BDT3 has been successfully authorized!


## 1. Intro to Uber H3.

* OSS Library developed at Uber.

* A hexogonal and hierarchical spatial grid.

* Can scale up and down resolutions easily.

* Increasingly popular across industry.

<img src="DevSummit23Images/uber_h3_ba.png" alt="Drawing" style="width: 400px;">

* BDT has Uber H3 Functionality

In [2]:
spark.sql("""
    SHOW USER FUNCTIONS LIKE 'H3*'
""").show()

+----------------+
|        function|
+----------------+
|      h3distance|
|         h3kring|
|h3kringdistances|
|    h3tochildren|
|         h3togeo|
| h3togeoboundary|
|      h3toparent|
|      h3tostring|
+----------------+



## 2. Intro to OLS.

* OLS is a type of Regression (Data Science Tool)

* Predicts a dependent variable from independent variables.

<!-- -->

* Parameters $\hat{\beta}_p$ (beta coefficients) and $\hat{\epsilon}_i$ (random error) are estimated.

<img src="DevSummit23Images/Regression.png" alt="Regression_Eq" style="width: 600px;">

* The regression will minimize the difference between predicted and actual.

<img src="DevSummit23Images/OLS.png" alt="Residuals" style="width: 400px;">

## 2. Intro to GWR.

* GWR runs seperate regressions over each geographic subset.

<!-- -->

* The input data is weighted geographically.

<!-- -->

* Weights are part of the model training process.

<!-- -->

* OLS is actually a weighted regression with all weights equal to 1.

<img src="DevSummit23Images/GWR2.png" alt="GWR" style="width: 600px;">

## 4. Why GWR in BDT with Uber H3?

* Increased customer demand for H3.

<!-- -->

* Increased customer demand for data science capabilities.

<!-- -->


* BDT can scale GWR efficiently for large datasets.

<!-- -->


* BDT runs in your cloud cluster environment.

## 5. GWR with H3 in BDT: Under the Hood

### 5.1 Parent GWR Area

#### 5.1.1 Visualization

* GWR Areas are determined by parent H3 hexbins.

<!-- -->
   
* Example - Obervation Unit: Res 7, Parent GWR Area: Res 6

<img src="DevSummit23Images/child_parent.png" alt="Drawing" style="width: 400px;">

#### 5.1.2 BDT Code

* Load in the sample data.

In [3]:
input_df = spark \
    .read \
    .parquet("/Users/ale10305/ws/bdt3/bdt3-py/src/test/python/bdt/resources/gwrh3_input.prq") \
    .selectExpr("h7_idx")

input_df.printSchema()

root
 |-- h7_idx: long (nullable = true)



* Getting at parent H3 Index is simple in BDT.

In [4]:
gwr_col_name = "gwr_parent_idx"

parent = input_df \
    .selectExpr("h7_idx", f"H3ToParent(h7_idx, 6) AS {gwr_col_name }")

parent.show(1)

+------------------+------------------+
|            h7_idx|    gwr_parent_idx|
+------------------+------------------+
|609196265925771263|604692666348732415|
+------------------+------------------+
only showing top 1 row



### 5.2 GWR K-Ring Neighbors

#### 5.2.1 Visualization

* The GWR area also includes the neighbors around it.

<!-- -->

* The number of rings (k) represents the bandwidth of the GWR.

<!-- -->

* Neighboring observations are weighted by k-ring association.

$$
  w(obs_k) = \frac{1}{1+k}
$$

<img src="DevSummit23Images/parent_nbrs_3.png" alt="Drawing" style="width: 500px;">

#### 5.2.2 BDT Code

* BDT has functionality to get H3 k-ring neighbors.

In [5]:
help(h3kRingDistances)

Help on function h3kRingDistances in module bdt.functions:

h3kRingDistances(h3Idx, k) -> pyspark.sql.column.Column
    Return a collection of collections of the H3 Indices within k distance of the origin H3 index.
    Use explode() to unpack.
    
    This function uses the Uber H3 Library.
    
    Args:
        h3Idx: The H3 index value.
        k: The distance from the origin H3 index.
    
    Returns:
        Array[Array[Long]
    
    SQL Example::
    
        spark.sql('''SELECT GeoToH3(90, 180, 10) AS h3Idx''').createOrReplaceTempView("df")
    
        spark.sql('''SELECT explode(H3kRingDistances(h3Idx, 10)) AS nbr_h3Indices FROM df''')
    
    Python Example::
    
        spark \
            .createDataFrame([(180, 90)], ["lon", "lat"]) \
            .select(geoToH3("lat", "lon", 10).alias("h3Idx"))
    
        df.select(explode(h3kRingDistances("h3Idx", 10)).alias("nbr_h3Indices"))



In [6]:
k = 6
weight_col_name = "weight"

nbr_df_gwr = parent \
        .select(col(gwr_col_name), explode(h3kRingDistances(gwr_col_name, k)).alias("knn")) \
        .select(col(gwr_col_name), col("knn"), floor(size(col("knn")) / 6).alias("h3_ring_idx")) \
        .select(col(gwr_col_name), col("knn"), (1 / (col("h3_ring_idx") + 1)).alias(weight_col_name)) \
        .select(col(gwr_col_name), explode(col("knn")).alias("gwr_k_ring_nbr"), col(weight_col_name))

nbr_df_gwr.show()

+------------------+------------------+------------------+
|    gwr_parent_idx|    gwr_k_ring_nbr|            weight|
+------------------+------------------+------------------+
|604692666348732415|604692666348732415|               1.0|
|604692666348732415|604692824054562815|               0.5|
|604692666348732415|604692823517691903|               0.5|
|604692666348732415|604692665811861503|               0.5|
|604692666348732415|604692665543426047|               0.5|
|604692666348732415|604692666080296959|               0.5|
|604692666348732415|604692670241046527|               0.5|
|604692666348732415|604692670106828799|0.3333333333333333|
|604692666348732415|604692823920345087|0.3333333333333333|
|604692666348732415|604692823383474175|0.3333333333333333|
|604692666348732415|604692823786127359|0.3333333333333333|
|604692666348732415|604692819625377791|0.3333333333333333|
|604692666348732415|604692819759595519|0.3333333333333333|
|604692666348732415|604692665946079231|0.333333333333333

## 4. Running GWR in BDT

* We will use BDT ProcessorGWRH3.

<!-- -->


* Python Statmodels library is used for regression.

<!-- -->

* Sample Data: Uber Hex Bins (Res 7) enriched with Esri Buisiness Analyst Data.

|Population | householdi | educationa |spendingto | psychograp |
|-------------------|---------|---------|-----------------------|------|
|the total population  | Median household income (annual)  | education level | annual budget expenditures             | have 1st home mortgage|

| n_visits_bi  |
|---|
| number of visits (annual)   |

In [7]:
input_data = spark \
    .read \
    .parquet("/Users/ale10305/Desktop/gwrh3_input_50_atl_shp.prq") \
    .repartition(12)

input_data.printSchema()

r_results = gwrh3(
    input_data,
    obs_col="h7_idx",
    dependent_col="n_visits_bi",
    independent_cols=["population", "householdi", "educationa", "spendingto", "psychograp"],
    gwr_res=5,
    k=3)

print(type(r_results))

root
 |-- h7_idx: long (nullable = true)
 |-- SHAPE: struct (nullable = true)
 |    |-- WKB: binary (nullable = true)
 |    |-- XMIN: double (nullable = true)
 |    |-- YMIN: double (nullable = true)
 |    |-- XMAX: double (nullable = true)
 |    |-- YMAX: double (nullable = true)
 |-- hasdata: long (nullable = true)
 |-- aggregatio: string (nullable = true)
 |-- population: double (nullable = true)
 |-- householdi: double (nullable = true)
 |-- educationa: double (nullable = true)
 |-- spendingto: double (nullable = true)
 |-- psychograp: double (nullable = true)
 |-- shape_leng: double (nullable = true)
 |-- shape_area: double (nullable = true)
 |-- n_factor: short (nullable = true)
 |-- n_visits_bi: long (nullable = true)
 |-- mid: long (nullable = true)

<class 'bdt.gwrh3.RegressionResultGWRH3'>


In [8]:
input_data.show(1)

+------------------+--------------------+-------+--------------------+----------+----------+----------+------------+----------+-------------+-------------+--------+-----------+---+
|            h7_idx|               SHAPE|hasdata|          aggregatio|population|householdi|educationa|  spendingto|psychograp|   shape_leng|   shape_area|n_factor|n_visits_bi|mid|
+------------------+--------------------+-------+--------------------+----------+----------+----------+------------+----------+-------------+-------------+--------+-----------+---+
|609195527057178623|{          ...|      1|BlockApportionmen...|    6431.0|   37353.0|     535.0|1.29250388E8|     816.0|9922.47716028|7088123.26873|    1158|       3765| 49|
+------------------+--------------------+-------+--------------------+----------+----------+----------+------------+----------+-------------+-------------+--------+-----------+---+
only showing top 1 row



## 5. Evaluating Model Perfomance

* Evaluate the trained model on the test dataset.

In [9]:
%%time
tested = r_results.eval_on_test()
tested.selectExpr("tval_spendingto", "pval_spendingto", "n_visits_bi AS y", "y_hat", "resid_y", "adj_r2").show(7)

+------------------+--------------------+-----+------------------+-------------------+------------------+
|   tval_spendingto|     pval_spendingto|    y|             y_hat|            resid_y|            adj_r2|
+------------------+--------------------+-----+------------------+-------------------+------------------+
|1.6673301997081775| 0.10041283549301706| 4646| 3233.280553624121|-1412.7194463758792|0.9327191117028645|
|3.6477953689973566|4.279489818228621E-4| 1018| 3149.855063653106|  2131.855063653106|0.9199393801673219|
| 5.797610998613623|2.174827536886712E-8|11239| 9673.067898797464|-1565.9321012025357|0.9449181123309583|
|  4.75488718177388|4.302928319930997E-6| 1180| 1729.871563015779|   549.871563015779|0.9329646139244013|
| 5.218477739590955|4.516344607356565E-7| 1610| 911.1056646826137| -698.8943353173863|0.9211921651661866|
|1.6673301997081775| 0.10041283549301706| 6092| 5292.709313537567| -799.2906864624329|0.9327191117028645|
| 5.218477739590955|4.516344607356565E-7|  115

* Model viablity stats:

    * pval: Low (< 0.05) indicates significance.
    
    
    <img src="DevSummit23Images/p-val.png" alt="Drawing" style="width: 600px;">



    * tval: A high value indicates significance.


    * r^2: Between 0 and 1. Measures goodness of fit.

* Unpersist the DataFrame.

In [10]:
r_results.unpersist()

DataFrame[gwr_idx: bigint, n_obs: bigint, beta_population: double, beta_householdi: double, beta_educationa: double, beta_spendingto: double, beta_psychograp: double, tval_population: double, tval_householdi: double, tval_educationa: double, tval_spendingto: double, tval_psychograp: double, pval_population: double, pval_householdi: double, pval_educationa: double, pval_spendingto: double, pval_psychograp: double, adj_r2: double, reg_type: string]

## Performance

### 5.1 Databricks

|Num Workers | Memory (Each) | Num Cores (Each) |
|------------|--------|-----------|
| 3          | 32G    |8          |

|Num Obs Units (H7) | GWR Res | k-rings |Size of Nbrs DataFrame | Time |
|-------------------|---------|---------|-----------------------|------|
| 3,937             | 5 (H5)  |4        |1,404,411              | 14 sec|

* The input DataFrame is exploded for regression:  3,937 to 1,404,411

<!-- -->

* Upscaling to resolution of 5 with 4 rings of neighbors explodes DF

## 6. Limitations and Future Improvements

* Using Pandas and Statmodels is not efficient.

* Writing the GWR H3 algorithm native to BDT.

* More extensive testing.

#### 7. Image Reference

* https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/regression-analysis-basics.htm

* https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/ordinary-least-squares.htm